In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
import transformers

In [ ]:
!pip install accelerate -U
!pip install datasets
!pip install evaluate -U
!pip install rouge_score
!pip install sentencepiece

In [2]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# import tensorflow as tf

# # Detect hardware
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     print(f"Running on TPU {tpu.master()}")
# except ValueError:
#     print("Not connected to a TPU runtime. Using CPU/GPU.")
#     tpu = None

# # Select appropriate distribution strategy
# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     strategy = tf.distribute.get_strategy()

# print(f"Number of accelerators: {strategy.num_replicas_in_sync}")

Running on TPU grpc://10.126.129.42:8470


Number of accelerators: 8


In [3]:
model_checkpoint = "google/mt5-small"

In [4]:
from torch.utils.data import Dataset, DataLoader

In [6]:
from datasets import load_dataset


raw_datasets =  load_dataset("toanduc/10k-final")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [7]:
from transformers import BertTokenizer,BertModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [8]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = "summarize "

In [ ]:
import re
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

In [9]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Original Text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    labels = tokenizer(text_target=examples["Summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/11585 [00:00<?, ? examples/s]

Map:   0%|          | 0/907 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, MT5Config

model = MT5ForConditionalGeneration.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
model_name = model_checkpoint.split("/")[-1]
fine_tuned_model_name = f"{model_name}-sport"

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
from evaluate import load
metric = load("rouge")

In [17]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    result = {key: value * 100 for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
num_epochs = 16
batch_size = 4

args = Seq2SeqTrainingArguments(
  fine_tuned_model_name,
  evaluation_strategy="epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=batch_size,
  per_device_eval_batch_size=batch_size,
  weight_decay=0.01,
  save_total_limit=3,
  num_train_epochs=num_epochs,
  predict_with_generate=True,
  fp16=False,
)


trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.308500,1.668016,21.962400,8.302800,16.862600,18.489200,19.000000
2,1.960100,1.512935,22.355000,8.097600,17.100100,18.451800,19.000000
3,1.814900,1.446059,21.755300,8.269300,16.881300,18.175300,19.000000
4,1.693900,1.385942,22.148500,8.544400,17.195700,18.564100,19.000000
5,1.666100,1.360043,22.146400,8.570100,17.191100,18.583400,19.000000
6,1.577600,1.336613,22.188000,8.610200,17.211400,18.617100,19.000000
7,1.563500,1.303611,22.221600,8.805300,17.325500,18.712700,19.000000
8,1.528600,1.296895,22.609800,9.461800,17.741700,19.234400,19.000000
9,1.503400,1.277390,22.721100,9.832300,17.970900,19.456000,19.000000
10,1.480800,1.269683,22.605700,9.778000,17.917600,19.359300,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=46352, training_loss=1.6960067597594308, metrics={'train_runtime': 32865.7957, 'train_samples_per_second': 5.64, 'train_steps_per_second': 1.41, 'total_flos': 1.8756971251501056e+17, 'train_loss': 1.6960067597594308, 'epoch': 16.0})

In [21]:
trainer.push_to_hub()

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

events.out.tfevents.1710205559.d5f8b6fd30a3.1294.0:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/toanduc/mt5-small-sport/commit/596ffdae9e95d1b2463611fe1779f5bb6c43030c', commit_message='End of training', commit_description='', oid='596ffdae9e95d1b2463611fe1779f5bb6c43030c', pr_url=None, pr_revision=None, pr_num=None)